<a href="https://colab.research.google.com/github/lauramoraes/Analise_de_Clusters/blob/master/2.1%20-%20Medidas%20de%20Similaridade%20e%20Dist%C3%A2ncia%20(Wine).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similaridade e distância

Utilizamos medidas de similaridade (ou distância) para quantificar o quanto dois objetos se parecem. Essa informação é útil em diversas aplicações, como:

1. Sistemas de recomendação
2. Agrupamentos
3. Classificação e regressão
4. Detecção de anomalias/outliers
5. Descoberta de casos parecidos (diagnóstico médico, precedentes legais)


### Importando bibliotecas a serem usadas

In [0]:
install.packages("proxy")
install.packages("gdata")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


In [1]:
suppressMessages(suppressWarnings(library(tidyverse)))
suppressMessages(suppressWarnings(library(proxy)))
suppressMessages(suppressWarnings(library(gdata)))

ERROR: ignored

# Whisky Analytics

Foram criadas caraterísticas numéricas que resumem as informações das notas de degustação. Foram definidos cinco atributos gerais de uísquem cada um com muitos valores:

<img src="https://drive.google.com/uc?export=view&id=1mN_UmQeD43_8WfA9hiIRANXLnzAcgKIM">


É importante notar que essas categorias de valores **não** são mutuamente exclusivas por exemplo, o paladar de Aberlour é descrito como médio, íntegro, suave, completo e macio). Em geral, qualquer um dos valores pode ocorrer ao mesmo tempo (embora alguns deles, como a cor ser clara e esfumaçada nunca acontece), mas como podem coocorrer, cada valor de cada variável foi codificado como uma característica separada.

### Carregando os dados salvos anteriormente

In [2]:
list.files()

[1] "data_cleaned.Rdata" "data_names.Rdata"   "sample_data"

In [3]:
vars = load("data_names.Rdata")
vars
vars = load("data_cleaned.Rdata")
vars
# vars = load("data_onehot.Rdata")
# vars
head(data_cleaned)

[1] "data_names"

[1] "data_cleaned"

Alcohol,Malic,Ash,Alcalinity,Magnesium,Phenols,Flavanoids,Nonflavanoids,Proanthocyanins,Color,Hue,Dilution,Proline
<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450


In [4]:
data_names

Type
<int>
1
1
1
1
1
1
1
1
1



### Calculando distâncias

In [5]:
# Adicionando coluna de nome
rownames(data_cleaned) <- data_names$NAME

# Selecionando alguns vinhos
names_list <- c("Bunnahabhain", "Glenglassaugh", "Tullibardine", "Ardberg", "Bruichladdich", "Glenmorangie")
data_selected <- data_cleaned[names_list,]

# Selecionando algumas propriedades
color <- c("gold", "pale", "sherry", "p.gold")
body <- c('soft','med', 'full', 'round', 'smooth', 'light', 'firm', 'oily')
palate <- c('full.1', 'dry', 'sherry.1', 'big', 'light.1', 'smooth.1', 'clean', 'fruit', 'grass', 'smoke', 'sweet', 'spice', 'oil', 'salt', 'arome')
nose <- c("AROMA", 'PEAT', 'SWEET', 'LIGHT', 'FRESH', 'DRY', 'FRUIT', 'GRASS', 'SEA', 'SHERRY', 'SPICY', 'RICH')
finish <- c('full.2', 'dry.1', 'warm', 'big.1', 'light.2', 'smooth.2', 'clean.1', 'fruit.1', 'grass.1', 'smoke.1', 'sweet.1', 'spice.1', 'oil.1', 'salt.1', 'arome.1', 'ling', 'long', 'very', 'quick')
attributes <- c(color, body, palate, nose, finish)
data_selected <- data_selected[,attributes]
data_selected

ERROR: ignored

Existem diversos tipos de distância. Quando os atributos são heterogêneos e categóricos, temos algumas alternativas:
1. Categorizá-los em números (como fizemos anteriormente)
2. Usar uma distância apropriada para dados categóricos (como a de Jaccard, por exemplo, utilizada nesse exemplo do Whisky).

Exemplos de distância são:
1. Distância Euclidiana: normalmente a mais utilizada. É geral, intuitiva e computacionalmente muito rápida.
1. Distância de Manhattan
2. Distância de coseno: útil quando se quer ignorar diferenças de escala, como o tamanho dos textos.
3. Distância de Jaccard: Trata dois objetos como conjuntos de características. Apropriado para problemas em que a posse de uma característica comum é importante, mas a ausência não.


In [16]:
dist(data_cleaned[1:8,], method="euclidean")

          1         2         3         4         5         6         7
2  31.26501                                                            
3 122.83115 135.22469                                                  
4 415.24540 430.25156 295.26271                                        
5 330.17450 315.66880 450.33105 745.03944                              
6 385.29596 400.21029 265.25695  30.09197 715.05460                    
7 227.12947 240.06341 105.21146 190.79623 555.47585 160.81214          
8 230.09160 245.98497 111.82768 185.20475 560.02040 155.29689  25.68653

In [0]:
pr_DB$get_entry("fJaccard")

      names fJaccard, fuzzy_Jaccard
        FUN R_fuzzy_dist
   distance TRUE
     PREFUN pr_fJaccard_prefun
    POSTFUN NA
    convert pr_dist2simil
       type metric
       loop FALSE
      C_FUN TRUE
    PACKAGE proxy
       abcd FALSE
    formula sum_i (min{x_i, y_i} / max{x_i, y_i})
  reference Miyamoto S. (1990). Fuzzy sets in information retrieval and
            cluster analysis, Kluwer Academic Publishers, Dordrecht.
description The fuzzy Jaccard dissimilarity (C implementation).